In [ ]:
import requests
from requests import get
#from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from urllib.request import urlopen
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
headers = {"Accept-Language": "en-US, en;q=0.5"}
opts = Options()
opts.headless = False
driver = webdriver.Chrome(ChromeDriverManager().install(),options=opts)

In [ ]:
page=1
while page!=26:
    url=f"https://ojireh.com/shop-now.html?p={page}"
    attempt = 1
    max_attempts=3
    try:
        driver.get(url)
        #driver.find_element_by_tag_name('html').send_keys(Keys.END)
    except TimeoutException as ex:
        print(ex.Message)
        driver.navigate().refresh()
     
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")#scrolls through page to ensure pge loads before scraping
    WebDriverWait(driver,5)
    try:  
        products= WebDriverWait(driver,30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"img.product-image-photo.default_image")))
        for p in products:
            if p is not None:
                file_name=p.get_attribute('alt').replace('/','-').strip()
                #specialChars = '!#"$:[]$@%^&*()'
                #for specialChar in specialChars:
                    #file_name=file_name.replace(specialChar, '')
            else:
                file_name=None
            print (file_name)
            if p is not None:
                #img = p.find_element_by_tag_name("img")
                img_src= p.get_attribute("src")
            else:
                img_src=None
            print(img_src)
            response = requests.get(img_src)
            file = open(f'{file_name}.jpg', "wb")
            file.write(response.content)
            file.close()
    except StaleElementReferenceException:
        if attempt == max_attempts:
                raise
        attempt += 1
    page=page+1    
    
    